In [55]:
import os
from pathlib import Path
print(Path().cwd())

/workspaces/Kidney-Disease-Classification/research


In [56]:
%pwd

'/workspaces/Kidney-Disease-Classification/research'

In [57]:
current_dir = Path().cwd()
current_dir = current_dir.parent
print(current_dir), current_dir

/workspaces/Kidney-Disease-Classification


(None, PosixPath('/workspaces/Kidney-Disease-Classification'))

In [58]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [59]:
from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from cnnClassifier.utils.common import *


class ConfigurationManager:
    def __init__(
        self,
        config_filepath = current_dir / CONFIG_FILE_PATH,
        params_filepath = current_dir / PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model

        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir = Path(config.root_dir),
            base_model_path = Path(config.base_model_path),
            updated_base_model_path = Path(config.updated_base_model_path),
            params_image_size = self.params.IMAGE_SIZE,
            params_learning_rate = self.params.LEARNING_RATE,
            params_include_top = self.params.INCLUDE_TOP,
            params_weights = self.params.WEIGHTS,
            params_classes = self.params.CLASSES
        )

        return prepare_base_model_config



In [60]:
import os
import urllib.request as request
from zipfile import ZipFile

In [61]:
import tensorflow as tf

In [62]:
print("Num CPUs Available: ", len(tf.config.list_physical_devices('CPU')))

Num CPUs Available:  1


In [63]:
config_manager = ConfigurationManager()

[2026-01-10 11:02:50,800: INFO: common: yaml file: /workspaces/Kidney-Disease-Classification/config/config.yaml loaded successfully]
[2026-01-10 11:02:50,804: INFO: common: yaml file: /workspaces/Kidney-Disease-Classification/params.yaml loaded successfully]
[2026-01-10 11:02:50,806: INFO: common: created directory at: artifacts]


In [64]:
config = config_manager.prepare_base_model_config()
print(config)

[2026-01-10 11:02:50,814: INFO: common: created directory at: artifacts/prepare_base_model]
PrepareBaseModelConfig(root_dir=PosixPath('artifacts/prepare_base_model'), base_model_path=PosixPath('artifacts/prepare_base_model/model.h5'), updated_base_model_path=PosixPath('artifacts/prepare_base_model/model_updated.h5'), params_image_size=BoxList([224, 224, 3]), params_learning_rate=0.01, params_include_top=False, params_weights='imagenet', params_classes=2)


In [65]:
vgg16_model = tf.keras.applications.VGG16(
        include_top=config.params_include_top,
        weights=config.params_weights,
        input_shape=config.params_image_size,
        name="vgg16",
        )

In [66]:
model = vgg16_model

In [67]:
model.layers

[<InputLayer name=input_layer_3, built=True>,
 <Conv2D name=block1_conv1, built=True>,
 <Conv2D name=block1_conv2, built=True>,
 <MaxPooling2D name=block1_pool, built=True>,
 <Conv2D name=block2_conv1, built=True>,
 <Conv2D name=block2_conv2, built=True>,
 <MaxPooling2D name=block2_pool, built=True>,
 <Conv2D name=block3_conv1, built=True>,
 <Conv2D name=block3_conv2, built=True>,
 <Conv2D name=block3_conv3, built=True>,
 <MaxPooling2D name=block3_pool, built=True>,
 <Conv2D name=block4_conv1, built=True>,
 <Conv2D name=block4_conv2, built=True>,
 <Conv2D name=block4_conv3, built=True>,
 <MaxPooling2D name=block4_pool, built=True>,
 <Conv2D name=block5_conv1, built=True>,
 <Conv2D name=block5_conv2, built=True>,
 <Conv2D name=block5_conv3, built=True>,
 <MaxPooling2D name=block5_pool, built=True>]

In [68]:
for i, layer in enumerate(model.layers):
    print(f"Is Layer - {i+1} trainable ? => {"yes" if layer.trainable else "No" }")

Is Layer - 1 trainable ? => yes
Is Layer - 2 trainable ? => yes
Is Layer - 3 trainable ? => yes
Is Layer - 4 trainable ? => yes
Is Layer - 5 trainable ? => yes
Is Layer - 6 trainable ? => yes
Is Layer - 7 trainable ? => yes
Is Layer - 8 trainable ? => yes
Is Layer - 9 trainable ? => yes
Is Layer - 10 trainable ? => yes
Is Layer - 11 trainable ? => yes
Is Layer - 12 trainable ? => yes
Is Layer - 13 trainable ? => yes
Is Layer - 14 trainable ? => yes
Is Layer - 15 trainable ? => yes
Is Layer - 16 trainable ? => yes
Is Layer - 17 trainable ? => yes
Is Layer - 18 trainable ? => yes
Is Layer - 19 trainable ? => yes


In [69]:
model.output

<KerasTensor shape=(None, 7, 7, 512), dtype=float32, sparse=False, ragged=False, name=keras_tensor_79>

In [70]:
print(model.input)

<KerasTensor shape=(None, 224, 224, 3), dtype=float32, sparse=False, ragged=False, name=keras_tensor_61>


In [71]:
7 * 7 * 512

25088

In [72]:
flatten_input  = tf.keras.layers.Flatten()(model.output)
flatten_input

<KerasTensor shape=(None, 25088), dtype=float32, sparse=False, ragged=False, name=keras_tensor_80>

In [73]:
prediction = tf.keras.layers.Dense(units = 2, activation='softmax')(flatten_input)
prediction

<KerasTensor shape=(None, 2), dtype=float32, sparse=False, ragged=False, name=keras_tensor_81>

In [74]:
full_model = tf.keras.models.Model(
    inputs = model.input,
    outputs = prediction
)

In [75]:
full_model

<Functional name=functional_2, built=True>

In [76]:
full_model.name = "Kidney_Disease_Classification_Model"

In [77]:
full_model

<Functional name=Kidney_Disease_Classification_Model, built=True>

SGD (Optimizer): Stochastic Gradient Descent with a set learning rate is chosen for stability. While newer optimizers (like Adam) are faster, SGD often generalizes better on medical imaging, preventing the model from overfitting to specific textures in the training scans.

Categorical Crossentropy (Loss): This is the standard for multi-class classification (e.g., Normal, Cyst, Tumor, Stone). It calculates the "distance" between the predicted probability and the actual label, penalizing wrong guesses heavily.

Accuracy (Metric): This provides an easy-to-read percentage of how many CT scans were correctly classified, which is the primary goal for medical screening tools.

In [78]:
full_model.compile(
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01),
    loss = tf.keras.losses.CategoricalCrossentropy(),
    metrics=["accuracy"]
)

In [79]:
full_model.summary()

Model: "Kidney_Disease_Classification_Model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │        50,178 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,764,866 (56.32 MB)

 Trainable params: 14,764,866 (56.32 MB)

 Non-trainable params: 0 (0.00 B)

In [80]:
config.updated_base_model_path

PosixPath('artifacts/prepare_base_model/model_updated.h5')

In [81]:
model.save(config.updated_base_model_path)

[2026-01-10 11:02:51,502: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]


In [84]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    def save_model(self, path:Path, model: tf.keras.Model):
        model.save(path)

    def get_base_model(self):
        self.model = tf.keras.applications.VGG16(
        include_top=self.config.params_include_top,
        weights=self.config.params_weights,
        input_shape=self.config.params_image_size,
        name="vgg16",
        )
        self.save_model(path=self.config.base_model_path, model=self.model)

    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif freeze_till is not None and freeze_till > 0:
            for layer in model.layers[:freeze_till]:
                model.trainable = False

        flatten_input = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(units = classes, activation='softmax')(flatten_input)

        full_model =  tf.keras.models.Model(
            inputs = model.input,
            outputs = prediction
        )

        full_model.compile(
           optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate),
           loss = tf.keras.losses.CategoricalCrossentropy(),
           metrics=["accuracy"]
        )

        full_model.summary()

        return full_model
    
    def update_base_model(self):
        self.full_model = PrepareBaseModel._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path = self.config.updated_base_model_path, model = self.full_model)

In [85]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2026-01-10 11:09:53,522: INFO: common: yaml file: /workspaces/Kidney-Disease-Classification/config/config.yaml loaded successfully]
[2026-01-10 11:09:53,526: INFO: common: yaml file: /workspaces/Kidney-Disease-Classification/params.yaml loaded successfully]
[2026-01-10 11:09:53,527: INFO: common: created directory at: artifacts]
[2026-01-10 11:09:53,529: INFO: common: created directory at: artifacts/prepare_base_model]
[2026-01-10 11:09:53,813: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 2)              │        50,178 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,764,866 (56.32 MB)

 Trainable params: 50,178 (196.01 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

[2026-01-10 11:09:53,982: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
